# Prep

In [31]:
print("Playground notebook")

Playground notebook


In [229]:
%notebook inline

In [230]:
from pymatgen.electronic_structure.boltztrap2 import *
from monty.serialization import loadfn
from pymatgen.electronic_structure.plotter import BSPlotter
from pymatgen.electronic_structure.plotter import DosPlotter

import matplotlib
orig_mpl_backend = plt.get_backend()
orig_mpl_backend = 'agg'
matplotlib.use('pgf')
plt.rcParams.update({
    "font.family": "serif",  # use serif/main font for text elements
    "text.usetex": True,     # use inline math for ticks
    "pgf.rcfonts": False,    # don't setup fonts from rc parameters
    "pgf.preamble": "\n".join([
         r"\usepackage{unicode-math}",   # unicode math setup
    ])
})

In [254]:
def fp_for_element(el):
    el = el.replace(' ', '_')
    if '-' in el:
        a, b = el.split('-')
        return f'{a}-defect-data/{b}/'
    
    return f'{el}/'

In [247]:
elements = {
    'Mo3Si2': ['MoSi', 'SiMo', 'VMo', 'VSi'],
    # 'MoSi2': ['MoSi', 'SiMo', 'VMo', 'VSi'],
    'MoSi2 tetragonal': [],
    'MoSi2 hexagonal': [],
}

In [248]:
# deviation from Fermi energy in eV
energy_range = 1.5
temp_r = np.arange(300,1300,300)
temps = [300, 600, 900, 1200]
dopings_r = 10.**np.arange(18,26,0.25)
dopings_arr = [1e22, 1e23]
doping_type = 'p'

save_to_disk = False
load_from_disk = not save_to_disk

partial_dos = False

In [94]:
class MyBztPlotter(BztPlotter):
    def plot_props(
        self,
        prop_y,
        prop_x,
        prop_z="temp",
        output="avg_eigs",
        dop_type="n",
        doping=None,
        temps=None,
        xlim=(-2, 2),
        ax=None,
        label="",
        linestyle="solid",
        marker="o",
        markevery=1,
        color="b",
    ):
        """
        Function to plot the transport properties.
        Args:
            prop_y: property to plot among ("Conductivity","Seebeck","Kappa","Carrier_conc",
                "Hall_carrier_conc_trace"). Abbreviations are possible, like "S" for "Seebeck"
            prop_x: independent variable in the x-axis among ('mu','doping','temp')
            prop_z: third variable to plot multiple curves ('doping','temp')
            output: 'avg_eigs' to plot the average of the eigenvalues of the properties
                tensors; 'eigs' to plot the three eigenvalues of the properties
                tensors.
            dop_type: 'n' or 'p' to specify the doping type in plots that use doping
                levels as prop_x or prop_z
            doping: list of doping level to plot, useful to reduce the number of curves
                when prop_z='doping'
            temps: list of temperatures to plot, useful to reduce the number of curves
                when prop_z='temp'
            xlim: chemical potential range in eV, useful when prop_x='mu'
            ax: figure.axes where to plot. If None, a new figure is produced.
        Example:
        bztPlotter.plot_props('S','mu','temp',temps=[600,900,1200]).show()
        more example are provided in the notebook
        "How to use Boltztra2 interface.ipynb".
        """
        props = (
            "Conductivity",
            "Seebeck",
            "Kappa",
            "Effective_mass",
            "Power_Factor",
            "Carrier_conc",
            "Hall_carrier_conc_trace",
        )
        props_lbl = (
            "Conductivity",
            "Seebeck",
            "$K_{el}$",
            "Effective mass",
            "Power Factor",
            "Carrier concentration",
            "Hall carrier conc.",
        )
        props_unit = (
            r"$(\mathrm{S\,m^{-1}})$",
            r"($\mu$V/K)",
            r"$(W / (m \cdot K))$",
            r"$(m_e)$",
            r"$( mW / (m\cdot K^2)$",
            r"$(cm^{-3})$",
            r"$(cm^{-3})$",
        )

        props_short = [p[: len(prop_y)] for p in props]

        if prop_y not in props_short:
            raise BoltztrapError("prop_y not valid")

        if prop_x not in ("mu", "doping", "temp"):
            raise BoltztrapError("prop_x not valid")

        if prop_z not in ("doping", "temp"):
            raise BoltztrapError("prop_z not valid")

        idx_prop = props_short.index(prop_y)

        leg_title = ""

        mu = self.bzt_transP.mu_r_eV

        if prop_z == "doping" and prop_x == "temp":
            p_array = eval("self.bzt_transP." + props[idx_prop] + "_" + prop_z)
        else:
            p_array = eval("self.bzt_transP." + props[idx_prop] + "_" + prop_x)

        if ax is None:
            plt.figure(figsize=(10, 8))

        temps_all = self.bzt_transP.temp_r.tolist()
        if temps is None:
            temps = self.bzt_transP.temp_r.tolist()

        if isinstance(self.bzt_transP.doping, np.ndarray):
            doping_all = self.bzt_transP.doping.tolist()
            if doping is None:
                doping = doping_all

        # special case of carrier and hall carrier concentration 2d arrays (temp,mu)
        if idx_prop in [5, 6]:
            if prop_z == "temp" and prop_x == "mu":
                for temp in temps:
                    ti = temps_all.index(temp)
                    prop_out = p_array[ti] if idx_prop == 6 else np.abs(p_array[ti])
                    plt.semilogy(mu, prop_out, label=label + ": " + str(temp) + " K", linestyle=linestyle, marker=marker, markevery=markevery, color=color)

                plt.xlabel(r"$\mu$ (eV)", fontsize=30)
                plt.xlim(xlim)
            else:
                raise BoltztrapError(
                    "only prop_x=mu and prop_z=temp are \
                    available for c.c. and Hall c.c.!"
                )

        elif prop_z == "temp" and prop_x == "mu":
            for temp in temps:
                ti = temps_all.index(temp)
                prop_out = np.linalg.eigh(p_array[ti])[0]
                if output == "avg_eigs":
                    plt.plot(mu, prop_out.mean(axis=1), label=label + ": " + str(temp) + " K", linestyle=linestyle, marker=marker, markevery=markevery, color=color)
                elif output == "eigs":
                    for i in range(3):
                        plt.plot(
                            mu,
                            prop_out[:, i],
                            label=label + ": " + "eig " + str(i) + " " + str(temp) + " K", linestyle=linestyle, marker=marker, markevery=markevery, color=color
                        )

            plt.xlabel(r"$\mu$ (eV)", fontsize=30)
            plt.xlim(xlim)

        elif prop_z == "temp" and prop_x == "doping":
            for temp in temps:
                ti = temps_all.index(temp)
                prop_out = np.linalg.eigh(p_array[dop_type][ti])[0]
                if output == "avg_eigs":
                    plt.semilogx(doping_all, prop_out.mean(axis=1), label=label + ": " + str(temp) + " K", linestyle=linestyle, marker=marker, color=color)
                elif output == "eigs":
                    for i in range(3):
                        plt.plot(
                            doping_all,
                            prop_out[:, i],
                            label=label + ": " + "eig " + str(i) + " " + str(temp) + " K", linestyle=linestyle, marker=marker, markevery=markevery, color=color
                        )
            plt.xlabel(r"Carrier conc. $cm^{-3}$", fontsize=30)
            leg_title = dop_type + "-type"

        elif prop_z == "doping" and prop_x == "temp":
            for dop in doping:
                di = doping_all.index(dop)
                prop_out = np.linalg.eigh(p_array[dop_type][:, di])[0]
                if output == "avg_eigs":
                    plt.plot(
                        temps_all,
                        prop_out.mean(axis=1),
                        label=label + ": " + str(dop) + " $cm^{-3}$", linestyle=linestyle, marker=marker, markevery=markevery, color=color
                    )
                elif output == "eigs":
                    for i in range(3):
                        plt.plot(
                            temps_all,
                            prop_out[:, i],
                            label=label + ": " + "eig " + str(i) + " " + str(dop) + " $cm^{-3}$", linestyle=linestyle, marker=marker, markevery=markevery, color=color
                        )

            plt.xlabel(r"Temperature (K)", fontsize=30)
            leg_title = dop_type + "-type"

        plt.ylabel(props_lbl[idx_prop] + " " + props_unit[idx_prop], fontsize=30)
        plt.xticks(fontsize=25)
        plt.yticks(fontsize=25)
        plt.legend(title=leg_title, fontsize=15)
        plt.tight_layout()
        plt.grid()
        return plt

In [256]:
def gather_element_data(el):
    vrun = Vasprun(f'{fp_for_element(el)}vasprun.xml', parse_projected_eigen=True)
    data = VasprunBSLoader(vrun)
    
    if save_to_disk:
        bztInterp = BztInterpolator(data,lpfac=10,energy_range=energy_range,curvature=True,save_bztInterp=True, fname=f'{fp_for_element(el)}bztInterp.json.gz')
    elif load_from_disk:
        bztInterp = BztInterpolator(data, load_bztInterp=True, fname=f'{fp_for_element(el)}bztInterp.json.gz')
        
    if save_to_disk:
        bztTransp = BztTransportProperties(bztInterp, temp_r=temp_r, doping=dopings_r, save_bztTranspProps=True, fname=f'{fp_for_element(el)}bztTranspProps.json.gz')
    elif load_from_disk:
        bztTransp = BztTransportProperties(bztInterp, load_bztTranspProps=True, fname=f'{fp_for_element(el)}bztTranspProps.json.gz')
        
    bztTransp.compute_properties_doping(doping=dopings_r)
    
    bztPlotter = MyBztPlotter(bztTransp,bztInterp)
    
    return {
        'data': data,
        'interp': bztInterp,
        'transp': bztTransp,
        'plotter': bztPlotter
    }

element_data = {}
for k, v in elements.items():
    for defect_name in v:
        el_data = gather_element_data(f'{k}-{defect_name}')
        element_data[f'{k}-{defect_name}'] = el_data
    
    el_data = gather_element_data(k)
    element_data[k] = el_data

/Users/max/opt/anaconda3/envs/boltztrap2/lib/python3.9/site-packages/pymatgen/io/vasp/inputs.py:1732: UserWarning: Ignoring unknown variable type SHA256
  warnings.warn(f"Ignoring unknown variable type {key}")
/Users/max/opt/anaconda3/envs/boltztrap2/lib/python3.9/site-packages/pymatgen/io/vasp/inputs.py:1732: UserWarning: Ignoring unknown variable type COPYR
  warnings.warn(f"Ignoring unknown variable type {key}")


In [227]:
def make_plt():
    return plt.figure(figsize=(10, 8))

def save_plt(ax, name):
    matplotlib.use('pgf')
    ax.savefig(f'figures/{name}.pdf', backend='pgf')
    ax.close()

# Plots

In [113]:
colors = ['#377eb8', '#ff7f00', '#4daf4a',
           '#f781bf', '#a65628', '#984ea3',
           '#999999', '#e41a1c', '#dede00']

linestyles = [
    ('solid', 'solid'),      # Same as (0, ()) or '-'
     ('dotted', 'dotted'),    # Same as (0, (1, 1)) or ':'
     ('dashed', 'dashed'),    # Same as '--'
     ('dashdot', 'dashdot'),  # Same as '-.'
     ('long dash with offset', (5, (10, 3))),
     ('densely dashed',        (0, (5, 1))),

     ('dashdotted',            (0, (3, 5, 1, 5))),
     ('densely dashdotted',    (0, (3, 1, 1, 1))),

     ('dashdotdotted',         (0, (3, 5, 1, 5, 1, 5))),
     ('loosely dashdotdotted', (0, (3, 10, 1, 10, 1, 10))),
     ('densely dashdotdotted', (0, (3, 1, 1, 1, 1, 1)))]

markers = ['o', 'v', 's',
          'X', 'd', '*',
          '2', 'x', 'P']

## Seebeck coefficient as function of...

In [259]:
ax = make_plt()
for i, k in enumerate(element_data.keys()):
    ax = element_data[k]['plotter'].plot_props('S','doping','temp',temps=temps[0:1],ax=ax,label=k,marker=markers[i],markevery=1450,color=colors[i],linestyle=linestyles[i][1])

save_plt(ax, "allmats_S_doping_temp")

In [261]:
ax = make_plt()
for i, k in enumerate(element_data.keys()):
    ax = element_data[k]['plotter'].plot_props('S','temp','doping',doping=dopings_arr[:1], dop_type=doping_type,ax=ax,label=k,marker=markers[i],markevery=1,color=colors[i],linestyle=linestyles[i][1])

save_plt(ax, "allmats_S_temp_doping")

In [262]:
ax = make_plt()
for i, k in enumerate(element_data.keys()):
    ax = element_data[k]['plotter'].plot_props('S','mu','temp',temps=temps[0:1],ax=ax,label=k,marker=markers[i],markevery=450,color=colors[i],linestyle=linestyles[i][1])

save_plt(ax, "allmats_S_mu_temp")

## Electronic conductivity as function of...

In [263]:
ax = make_plt()
for i, k in enumerate(element_data.keys()):
    ax = element_data[k]['plotter'].plot_props('C','doping','temp',temps=temps[0:1],ax=ax,label=k,marker=markers[i],markevery=150,color=colors[i],linestyle=linestyles[i][1])

save_plt(ax, "allmats_C_doping_temp")

In [265]:
ax = make_plt()
for i, k in enumerate(element_data.keys()):
    ax = element_data[k]['plotter'].plot_props('C','temp','doping',doping=dopings_arr[:1], dop_type=doping_type,ax=ax,label=k,marker=markers[i],markevery=50,color=colors[i],linestyle=linestyles[i][1])

save_plt(ax, "allmats_C_temp_doping")

In [266]:
ax = make_plt()
for i, k in enumerate(element_data.keys()):
    ax = element_data[k]['plotter'].plot_props('C','mu','temp',temps=temps[0:1],ax=ax,label=k,marker=markers[i],markevery=550,color=colors[i],linestyle=linestyles[i][1])

save_plt(ax, "allmats_C_mu_temp")

## Power factor as function of...

In [169]:
ax = make_plt()
for i, k in enumerate(element_data.keys()):
    ax = element_data[k]['plotter'].plot_props('Po','doping','temp',temps=temps[0:1],ax=ax,label=k,marker=markers[i],markevery=150,color=colors[i],linestyle=linestyles[i][1])

save_plt(ax, "allmats_Po_doping_temp")

In [170]:
ax = make_plt()
for i, k in enumerate(element_data.keys()):
    ax = element_data[k]['plotter'].plot_props('Po','temp','doping',doping=dopings_arr[:1], dop_type=doping_type,ax=ax,label=k,marker=markers[i],markevery=1,color=colors[i],linestyle=linestyles[i][1])

save_plt(ax, "allmats_Po_temp_doping")

In [240]:
ax = make_plt()
for i, k in enumerate(element_data.keys()):
    ax = element_data[k]['plotter'].plot_props('Po','mu','temp',temps=temps[0:1],ax=ax,label=k,marker=markers[i],markevery=350,color=colors[i],linestyle=linestyles[i][1])

save_plt(ax, "Po_mu_temp")

## Effective mass as function of...

In [172]:
ax = make_plt()
for i, k in enumerate(element_data.keys()):
    ax = element_data[k]['plotter'].plot_props('E','doping','temp',temps=temps[0:1],ax=ax,label=k,marker=markers[i],color=colors[i],linestyle=linestyles[i][1])

save_plt(ax, "allmats_E_doping_temp")

In [173]:
ax = make_plt()
for i, k in enumerate(element_data.keys()):
    ax = element_data[k]['plotter'].plot_props('E','temp','doping',doping=dopings_arr[:1], dop_type=doping_type,ax=ax,label=k,marker=markers[i],markevery=1,color=colors[i],linestyle=linestyles[i][1])

save_plt(ax, "allmats_E_temp_doping")

In [174]:
ax = make_plt()
for i, k in enumerate(element_data.keys()):
    ax = element_data[k]['plotter'].plot_props('E','mu','temp',temps=temps[0:1],ax=ax,label=k,marker=markers[i],markevery=460,color=colors[i],linestyle=linestyles[i][1])

save_plt(ax, "allmats_E_mu_temp")

# Other

In [175]:
vrun = Vasprun('vasprun.xml',parse_projected_eigen=True)
data = VasprunBSLoader(vrun)

FileNotFoundError: [Errno 2] No such file or directory: 'vasprun.xml'

In [ ]:
# set curvature=False to speed up in case you do not need effective mass or hall coeficients
if save_to_disk:
    bztInterp = BztInterpolator(data,lpfac=10,energy_range=energy_range,curvature=True,save_bztInterp=True, fname='bztInterp.json.gz')
    print("Saved to disk!")
elif load_from_disk:
    bztInterp = BztInterpolator(data, load_bztInterp=True, fname='bztInterp.json.gz')
    print("Loaded from disk!")

In [ ]:
# sbs = bztInterp.get_band_structure()
# list(sbs.bands.values())[0].shape
# BSPlotter(sbs).show(ylim=[-energy_range, energy_range])

In [ ]:
kpaths = [["Γ", "Y", "C0"], ["Σ0", "Γ", "Z", "A0"], ["E0", "T", "Y"], ["Γ", "S", "R", "Z", "T"]]

kp_lbl = {"A0":np.array([0.5000000000, 0.5000000000, 0.5000000000]),
          "A0'":np.array([-0.5000000000, -0.5000000000, -0.5000000000]),
          "C0":np.array([-0.5000000000, 0.5000000000, 0.0000000000]),
          "C0'":np.array([0.5000000000, -0.5000000000, -0.0000000000]),
          "E0":np.array([-0.5000000000, 0.5000000000, 0.5000000000]),
          "E0'":np.array([0.5000000000, -0.5000000000, -0.5000000000]),
          "Γ":np.array([0.0000000000, 0.0000000000, 0.0000000000]),
          "R":np.array([0.0000000000, 0.5000000000, 0.5000000000]),
          "R'":np.array([-0.0000000000, -0.5000000000, -0.5000000000]),
          "S":np.array([0.0000000000, 0.5000000000, 0.0000000000]),
          "S'":np.array([-0.0000000000, -0.5000000000, -0.0000000000]),
          "Σ0":np.array([0.5000000000, 0.5000000000, 0.0000000000]),
          "Σ0'":np.array([-0.5000000000, -0.5000000000, -0.0000000000]),
          "T":np.array([-0.5000000000, 0.5000000000, 0.5000000000]),
          "T'":np.array([0.5000000000, -0.5000000000, -0.5000000000]),
          "Y":np.array([-0.5000000000, 0.5000000000, 0.0000000000]),
          "Y'":np.array([0.5000000000, -0.5000000000, -0.0000000000]),
          "Z":np.array([0.0000000000, 0.0000000000, 0.5000000000]),
          "Z'":np.array([-0.0000000000, -0.0000000000, -0.5000000000]),
          }
sbs = bztInterp.get_band_structure(kpaths,kp_lbl)
BSPlotter(sbs).show(ylim=[-energy_range, energy_range])

In [ ]:
tot_dos = bztInterp.get_dos()

In [ ]:
if partial_dos:
    tot_proj_dos = bztInterp.get_dos(partial_dos=True, progress=True)

    pltdos = DosPlotter(sigma=0.004)
    pltdos.add_dos_dict(tot_proj_dos.get_element_dos())
    pltdos.show()

In [ ]:
if save_to_disk:
    bztTransp = BztTransportProperties(bztInterp, temp_r=temp_r, doping=dopings_r, save_bztTranspProps=True, fname='bztTranspProps.json.gz')
    print("Saved to disk!")
elif load_from_disk:
    bztTransp = BztTransportProperties(bztInterp, load_bztTranspProps=True, fname='bztTranspProps.json.gz')
    print("Loaded from disk!")

In [ ]:
print('\t'.join(['Temp', '\mu', 'rows', 'columns tensor']))
for p in bztTransp.Conductivity_mu, bztTransp.Seebeck_mu, bztTransp.Kappa_mu, \
         bztTransp.Effective_mass_mu, bztTransp.Power_Factor_mu, bztTransp.Carrier_conc_mu:
    print('\t'.join([str(i) for i in p.shape]))

In [ ]:
bztTransp.compute_properties_doping(doping=dopings_r)

In [ ]:
print('\t'.join(['Temp', 'Doping', 'rows', 'columns tensor']))
for p in bztTransp.Conductivity_doping, bztTransp.Seebeck_doping, bztTransp.Kappa_doping, \
         bztTransp.Carriers_conc_doping,bztTransp.Effective_mass_doping, bztTransp.Power_Factor_doping:
    print('\t'.join([str(i) for i in p[doping_type].shape]))

## Plots!

In [ ]:
bztPlotter = BztPlotter(bztTransp,bztInterp)

In [ ]:
ax = bztPlotter.plot_props('C','doping','temp',temps=temps[0:1])

ax = bztPlotter.plot_props('C','doping','temp',temps=temps[1:2],ax=ax)

In [ ]:
bztPlotter.plot_props('C','temp','doping',doping=dopings_arr, dop_type=doping_type).show()

In [ ]:
bztPlotter.plot_props('C','mu','temp',temps=temps)

In [ ]:
bztPlotter.plot_props('S','mu','temp',temps=temps)

In [ ]:
bztPlotter.plot_props('S','doping','temp', temps=temps, dop_type=doping_type).show()

In [ ]:
bztPlotter.plot_props('S','temp','doping',doping=dopings_arr, dop_type=doping_type).show()

In [ ]:
bztPlotter.plot_props('Po','doping','temp',temps=temps,dop_type=doping_type).show()

In [ ]:
bztPlotter.plot_props('Po','temp','doping',doping=dopings_arr, dop_type=doping_type).show()

In [ ]:
bztPlotter.plot_props('Po','mu','temp',temps=temps,dop_type=doping_type).show()

In [ ]:
bztPlotter.plot_props('E','doping','temp',temps=temps).show()

In [ ]:
bztPlotter.plot_props('E','temp','doping',doping=dopings_arr, dop_type=doping_type).show()

In [ ]:
bztPlotter.plot_props('E','mu','temp',temps=temps).show()

## Misc plots

In [ ]:
bztPlotter.plot_props('K','mu','temp',temps=[600,900,1200]).show()

In [ ]:
bztPlotter.plot_props('H','mu','temp',temps=[600,900,1200]).show()

In [ ]:
bztPlotter.plot_props('Ca','mu','temp',temps=[600,900,1200]).show()

In [ ]:
bztPlotter.plot_bands().show()

In [ ]:
bztPlotter.plot_dos(T=200).show()